In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def numeric_plot(df, df_numeric):
    for f in df_numeric:
        fig, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (0.15, 0.85)})
        ax_box.grid(alpha=0.75)
        ax_hist.grid(alpha=0.75)
        flierprops = dict(
            markerfacecolor="r", markersize=2, linestyle="none", markeredgecolor="r"
        )
        sns.boxplot(df[f], orient="h", ax=ax_box, flierprops=flierprops)
        sns.histplot(df[f], ax=ax_hist, bins=50, kde=True)

        # Remove x axis name for the boxplot
        ax_box.set(xlabel="")
        plt.show()
        
def clean_outliers(df, df_numeric):
    print(len(df))
    for f in df_numeric:
        q1 = df[f].quantile(0.25)
        q3 = df[f].quantile(0.75)
        iqr = q3 - q1
        upper_limit = q3 + iqr *1.5
        lower_limit = q1 - iqr *1.5
        new_df = df.loc[((df[f]<upper_limit) & (df[f]>lower_limit))]
    print(len(new_df), len(df)-len(new_df))
    df = new_df

def random_search_cv(x_train, y_train, x_test, y_test, model_params):
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.metrics import r2_score
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for model_name, mp in model_params.items():
        clf =  RandomizedSearchCV(mp['model'], mp['params'], return_train_score=False ,n_iter = 10, cv = cv, verbose=2, random_state=42, scoring=r2_score)
        clf.fit(x_train, y_train)
        y_predd = clf.predict(x_test)
        scores.append({
            'model': model_name,
            'best_score': r2_score(y_test, y_predd),
            'best_params': clf.best_params_
        })  
    df_2 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
    df_2
